In [ ]:
'''
中正漢學研究
http://140.123.13.91/journal/journal.html
'''

'''
匯入套件
'''
# 請求套件
import requests as req

# 格式化輸出工具
from pprint import pprint as pp

# HTML Parser
from bs4 import BeautifulSoup as bs

# 強制等待 (執行期間休息一下)
from time import sleep

# 整理 json 使用的工具
import json

# regular expression 工具
import re

# 子處理程序，用來取代 os.system 的功能
import subprocess

# 建立隨機數
from random import randint

# 資料庫 (sqlite3)
import sqlite3

# Excel 工具
from openpyxl import load_workbook
from openpyxl import Workbook

# 時間工具
from datetime import datetime

# 其它
import json, os, sys

'''設定'''
# 主要首頁
prefix = 'http://140.123.13.91/journal/'
url = prefix + 'journal.html'

# JSON 存檔路徑
json_path = "./中正漢學研究.json"

# 自訂標頭
my_headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36'
}

# 整理資料用的變數
listData = []

# 建立儲存檔案用的資料夾，不存在就新增
folderPath = '中正漢學研究'
if not os.path.exists(folderPath):
    os.makedirs(folderPath)

    
'''程式區域'''
# 取得主要頁面所有連結資訊
def getMainLinks():
    # 建立 parser
    res = req.get(url, headers = my_headers)
    res.encoding = 'utf-8'
    soup = bs(res.text, "lxml")
    
    # 取得所有連結
    a_elms = soup.select('table td.tdJournal > a')
    for a in a_elms:        
        listData.append({
            "title": a.get_text(),
            "link": f"{prefix}{a['href']}"
        })

# 取得內頁連結
def getSubLinks():
    # 走訪所有連結
    for index in range( len(listData) ):
        # 沒有 sub 屬性，則建立，為了放置細節頁的內容
        if "sub" not in listData[index]:
            listData[index]['sub'] = []
            
        #建立 parser
        res = req.get(listData[index]['link'], headers = my_headers)
        res.encoding = 'utf-8'
        soup = bs(res.text, "lxml")

        # 取得有 table 元素的所有連結資訊
        tr_elms = soup.select('table tr')
        if len(tr_elms) > 0:
            # 走訪每一個 tr
            for tr in tr_elms:
                # td 底下確實有 a
                if len(tr.select('td a')) > 0:
                    # 資料清理
                    a = tr.select('td')[0].select_one('a')
                    sub_title = a.get_text()

                    # 作者資訊
                    author = tr.select('td')[1].get_text()
                    
                    # 整理超連結格式
                    if re.search(r"^https?:\/\/", a['href']):
                        sub_link = f"{a['href']}"
                    else:
                        sub_link = f"{prefix}{a['href']}"
                    
                    # 整合資料
                    listData[index]['sub'].append({
                        "sub_title": re.sub(r"\n|\s", "", sub_title),
                        "sub_link": sub_link,
                        "author": re.sub(r"\n|\s", "", author)
                    })
        else: 
            # 取得 p 元素底下的所有 a 連結資訊
            a_elms = soup.select('p a')
            if len(a_elms) > 0:
                # 走訪所有 a
                for a in a_elms:
                    # 若 a 有 href 屬性
                    if a.has_attr('href'):
                        # 取得超連結
                        href = a['href']

                        # 判斷超連結是否有 article 字眼，代表可能是 pdf 連結
                        if re.search(r"article", href):
                            # 取得超連結內文
                            sub_title = a.get_text()

                            # 整理超連結格式
                            sub_link = f"{prefix}{a['href']}"

                            # 整合資料
                            listData[index]['sub'].append({
                                "sub_title": sub_title,
                                "sub_link": sub_link,
                                "author": ""
                            })      
                
# 儲存 JSON
def saveToJson():
    with open(f"{folderPath}/{json_path}", "w", encoding='utf-8') as fp:
        fp.write( json.dumps(listData, ensure_ascii=False, indent=4) )
        
# 將 JSON 資料寫進資料庫
def saveToDB():
    # 資料庫連線
    conn = sqlite3.connect("journals.db")
    
    try:
        # 預設空字串
        strJson = ''
        with open(json_path, "r", encoding='utf-8') as fp:
            # 取得 JSON 字串
            strJson = fp.read()

        # json 字串轉 dict
        listResult = json.loads(strJson)
        
        # 建立 cursor
        cursor = conn.cursor()

        # 取得資料總數
        sql_insert = f'''
        INSERT INTO `140_123_13_91_journals` (`title`, `link`, `sub_title`, `sub_link`, `author`, `created_at`)
        VALUES (?,?,?,?,?,?)
        '''

        # 取得資料
        for index in range( len(listResult) ):
            title = listResult[index]['title']
            link = listResult[index]['link']
            for idx in range( len(listResult[index]['sub']) ):
                sub_title = listResult[index]['sub'][idx]['sub_title']
                sub_link = listResult[index]['sub'][idx]['sub_link']
                author = listResult[index]['sub'][idx]['author']
                created_at = datetime.today().strftime("%Y-%m-%d %H-%M-%S")
                
                # 新增資料
                cursor.execute(sql_insert, (title, link, sub_title, sub_link, author, created_at))
                
        # 執行 SQL
        conn.commit()      
    except:
        # SQL 執行失敗時回滾
        conn.rollback()
        print("Unexpected error: ", sys.exc_info())
        
    # 關閉資料庫
    conn.close()

# 將 JSON 資料儲存到 Excel
def saveToExcel():
    # 動態新增檔案
    workbook = Workbook()
    worksheet = workbook.create_sheet("中正漢學研究", 0)
    
    # 取得主要的 sheet
    worksheet = workbook['中正漢學研究']

    # 預設空字串
    strJson = ''
    with open(f"{folderPath}/{json_path}", "r", encoding='utf-8') as fp:
        # 取得 JSON 字串
        strJson = fp.read()

    # json 字串轉 dict
    listResult = json.loads(strJson)
    
    # 設定欄位名稱
    worksheet['A1'] = 'id'
    worksheet['B1'] = 'title'
    worksheet['C1'] = 'link'
    worksheet['D1'] = 'sub_title'
    worksheet['E1'] = 'sub_link'
    worksheet['F1'] = 'author'
    worksheet['G1'] = 'created_at'
    
    # 列數
    row = 0
    
    # 取得資料
    for index in range( len(listResult) ):
        title = listResult[index]['title']
        link = listResult[index]['link']
        for idx in range( len(listResult[index]['sub']) ):
            sub_title = listResult[index]['sub'][idx]['sub_title']
            sub_link = listResult[index]['sub'][idx]['sub_link']
            author = listResult[index]['sub'][idx]['author']
            created_at = datetime.today().strftime("%Y-%m-%d %H-%M-%S")
            
            # 寫入 excel
            worksheet['A' + str(row + 2)] = str(row + 1)
            worksheet['B' + str(row + 2)] = title
            worksheet['C' + str(row + 2)] = link
            worksheet['D' + str(row + 2)] = sub_title
            worksheet['E' + str(row + 2)] = sub_link
            worksheet['F' + str(row + 2)] = author
            worksheet['G' + str(row + 2)] = created_at
            
            # 遞增數量，以便後續對應列數寫入
            row += 1
            
    # 儲存 workbook
    exportFile = f"{folderPath}/中正漢學研究.xlsx"
    workbook.save(exportFile)

    # 關閉 workbook
    workbook.close()

# 下載 PDF
def downloadPDF():
    with open(f"{folderPath}/{json_path}", "r", encoding='utf-8') as fp:
        #取得 json 字串
        strJson = fp.read()
        
        # 將 json 轉成 list (裡面是 dict 集合)
        listResult = json.loads(strJson)
        
        # 下載所有檔案
        for obj in listResult:
            # 每個 dict 底下，有個 key 叫作 sub，值裡面也是 dict
            for o in obj['sub']:
                # 下載 PDF
                os.system(f"curl {o['sub_link']} -o {folderPath}/{o['sub_title']}.pdf")
    
'''
執行區域
'''
if __name__ == "__main__":
    getMainLinks()
    getSubLinks()
    saveToJson()
#     saveToDB()
#     saveToExcel()
#     downloadPDF()